<a href="https://colab.research.google.com/github/twoheartKES/weather-sales-forecast-ai/blob/main/weather_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Process weekly disease data from CSV files for the years 2023, 2024, and 2025, and for specified regions (Seoul, Busan, Daegu, Incheon, Gwangju, Daejeon, Ulsan, Sejong, Gyeonggi, Gangwon, Chungbuk, Chungnam, Jeonbuk, Jeonnam, Gyeongbuk, Gyeongnam, Jeju) located in the directory `/content/drive/MyDrive/AI기상-질병데이터/`. For each file:
1.  Load the weekly data into a pandas DataFrame.
2.  Convert the weekly aggregated confirmed cases (from the `DATAARRTXT` column) into daily entries.
3.  Filter the daily data based on specific date ranges for each year: '2023-01-01' to '2023-12-30' for 2023, '2023-12-31' to '2024-12-28' for 2024, and '2024-12-29' to '2025-12-27' for 2025.
4.  Calculate evenly distributed daily confirmed cases by grouping by disease and week, summing weekly cases, and dividing by 7.
5.  Construct a final DataFrame with columns 'date', 'disease_subtitle', 'disease_title', 'region', and 'confirmed_cases' (representing the evenly distributed daily cases).
6.  Save the processed daily data to a new CSV file in the format `/content/drive/MyDrive/AI기상-질병데이터/{year}_지역별질병데이터_일별전처리_{region}.csv`.

## Initialize Parameters and Date Ranges

### Subtask:
### 1. 파라미터 및 기간 설정
- 분석 대상 연도(2023–2025)와 경기·서울 지역 코드 목록 정의
- 연도별 일자 범위(date_ranges) 지정
- 입력/출력용 Google Drive 경로 설정



**Reasoning**:
I need to define lists for years and regions, a dictionary for date ranges, and string variables for input/output directories as specified in the subtask instructions.



In [ ]:
years = [2023, 2024, 2025]
regions = [
    '경기_가평군_075',
    '경기_고양시_덕양구_076',
    '경기_고양시_일산동구_077',
    '경기_고양시_일산서구_078',
    '경기_과천시_079',
    '경기_광명시_080',
    '경기_광주시_081',
    '경기_구리시_082',
    '경기_군포시_083',
    '경기_김포시_084',
    '경기_남양주시_085',
    '경기_동두천시_086',
    '경기_부천시_130',
    '경기_부천시_소사구_087',
    '경기_부천시_오정구_088',
    '경기_성남시_분당구_090',
    '경기_성남시_수정구_091',
    '경기_성남시_중원구_092',
    '경기_수원시_권선구_093',
    '경기_수원시_영통구_094',
    '경기_수원시_장안구_095',
    '경기_수원시_팔달구_096',
    '경기_시흥시_097',
    '경기_안산시_단원구_098',
    '경기_안산시_상록구_099',
    '경기_안성시_100',
    '경기_안양시_동안구_101',
    '경기_안양시_만안구_102',
    '경기_양주시_103',
    '경기_양평군_104',
    '경기_여주시_105',
    '경기_연천군_106',
    '경기_오산시_107',
    '경기_용인시_기흥구_108',
    '경기_용인시_수지구_109',
    '경기_용인시_처인구_110',
    '경기_의왕시_111',
    '경기_의정부시_112',
    '경기_이천시_113',
    '경기_파주시_114',
    '경기_평택시_115',
    '경기_포천시_116',
    '경기_하남시_117',
    '경기_화성시_118',
    '서울_010',
]

date_ranges = {
    2023: {'start_date': '2023-01-01', 'end_date': '2023-12-30'},
    2024: {'start_date': '2023-12-31', 'end_date': '2024-12-28'},
    2025: {'start_date': '2024-12-29', 'end_date': '2025-12-06'}
}
base_input_dir = '/content/drive/MyDrive/AI기상-질병데이터/'
base_output_dir = '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/'

print("Parameters and date ranges initialized successfully.")

Parameters and date ranges initialized successfully.


## Process All Files in Loops

### Subtask:
각 연도와 지역을 순회하면서, 매 파일마다 주간 데이터를 불러오고 이를 일별 데이터로 변환한 뒤, 지정된 날짜 범위로 필터링하고, 주간 확진자 수를 일별로 균등 분배하여 최종 DataFrame을 구성하고 저장한다.


**Reasoning**:

각 연도와 지역을 대상으로 중첩 반복문을 구현해, Subtask에서 제시한 상세 절차에 따라 파일을 처리해야 한다. 이 과정에는 주간 데이터 로딩, 일별 데이터로의 변환, 특정 날짜 범위 필터링, 일별 균등 분배 확진자 수 계산, 최종 DataFrame 생성 및 저장이 포함된다. 또한 파일 입출력 과정에서 오류에 대비해 try-except 블록을 함께 사용한다



In [ ]:
import pandas as pd

date_ranges = {
    2023: {'start_date': '2023-01-01', 'end_date': '2023-12-30'},
    2024: {'start_date': '2023-12-31', 'end_date': '2024-12-28'},
    2025: {'start_date': '2024-12-29', 'end_date': '2025-12-06'}
}

for year in years:
    yearly_records = []  # ⭐ 연도별로 누적

    for region in regions:
        input_file_path = f"{base_input_dir}{year}년확진자/kdca_{year}_week_{region}.csv"

        try:
            df = pd.read_csv(input_file_path, encoding='utf-8-sig')
            print(f"Loaded: {input_file_path}")

            # 연도 첫 일요일
            year_start = pd.to_datetime(f"{year}-01-01")
            first_sunday = year_start + pd.Timedelta(days=(6 - year_start.weekday()) % 7)

            week_columns = [
                col for col in df.columns
                if col.startswith('COLUMN') and col != 'COLUMN1'
            ]

            for _, row in df.iterrows():
                disease_subtitle = row['SUBTITLE']

                for week_idx, week_col in enumerate(week_columns):
                    weekly_cases = row[week_col]

                    if pd.isna(weekly_cases):
                        weekly_cases = 0

                    daily_cases = weekly_cases / 7
                    week_start_date = first_sunday + pd.Timedelta(days=week_idx * 7)

                    for day_offset in range(7):
                        record_date = week_start_date + pd.Timedelta(days=day_offset)

                        yearly_records.append({
                            'date': record_date,
                            'region': region,
                            'disease_subtitle': disease_subtitle,
                            'confirmed_cases': daily_cases
                        })

        except FileNotFoundError:
            print(f"File not found: {input_file_path}")
        except Exception as e:
            print(f"Error processing {year} {region}: {e}")

    # ===============================
    # 연도별 DataFrame 생성 & 필터
    # ===============================
    df_yearly = pd.DataFrame(yearly_records)

    current_range = date_ranges[year]
    df_yearly = df_yearly[
        (df_yearly['date'] >= current_range['start_date']) &
        (df_yearly['date'] <= current_range['end_date'])
    ]

    # 정렬 (분석용 필수)
    df_yearly = df_yearly.sort_values(
        by=['date', 'region', 'disease_subtitle']
    ).reset_index(drop=True)

    # ===============================
    # 저장
    # ===============================
    output_file_path = (
        f"{base_output_dir}{year}년_일별__지역별_확진자.csv"
    )

    df_yearly.to_csv(output_file_path, index=False, encoding='utf-8')
    print(f"✅ Saved yearly file: {output_file_path}")


Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_가평군_075.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_고양시_덕양구_076.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_고양시_일산동구_077.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_고양시_일산서구_078.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_과천시_079.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_광명시_080.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_광주시_081.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_구리시_082.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_군포시_083.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_김포시_084.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_남양주시_085.csv
Loaded: /content/drive/MyDrive/AI기상-질병데이터/2023년확진자/kdca_2023_week_경기_동두천시_086.csv
Loaded: /con

In [27]:
import pandas as pd
import os

# ===============================
# 1. 경로 설정
# ===============================
disease_dir = '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/'
weather_dir = '/content/drive/MyDrive/AI기상-질병데이터/2023-2025기상데이터/'
output_dir = '/content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/'

os.makedirs(output_dir, exist_ok=True)

# ===============================
# 2. 날짜 범위 정의
# ===============================
date_ranges = {
    2023: {'start_date': '2023-01-01', 'end_date': '2023-12-30'},
    2024: {'start_date': '2023-12-31', 'end_date': '2024-12-28'},
    2025: {'start_date': '2024-12-29', 'end_date': '2025-12-06'}
}

# ===============================
# 3. 지역 → 지점명 매핑
# ===============================
mapping_regions = {
    '동두천': [
        '경기_구리시_082','경기_부천시_오정구_088','경기_과천시_079','경기_부천시_130',
        '경기_광명시_080','경기_부천시_소사구_087','경기_고양시_덕양구_076',
        '경기_의정부시_112','경기_고양시_일산동구_077','경기_고양시_일산서구_078',
        '경기_하남시_117','경기_남양주시_085','경기_김포시_084','경기_양주시_103',
        '경기_파주시_114','경기_동두천시_086',
    ],
    '수원': [
        '경기_수원시_팔달구_096','경기_수원시_장안구_095','경기_수원시_권선구_093',
        '경기_화성시_118','경기_수원시_영통구_094','경기_군포시_083',
        '경기_용인시_기흥구_108','경기_의왕시_111','경기_성남시_수정구_091',
        '경기_안산시_단원구_098','경기_성남시_중원구_092','경기_오산시_107',
        '경기_안산시_상록구_099','경기_안양시_동안구_101','경기_용인시_수지구_109',
        '경기_시흥시_097','경기_안양시_만안구_102','경기_성남시_분당구_090',
        '경기_용인시_처인구_110','경기_평택시_115',
    ],
    '양평': ['경기_양평군_104','경기_광주시_081','경기_가평군_075'],
    '이천': ['경기_이천시_113','경기_여주시_105','경기_안성시_100'],
    '파주': ['경기_포천시_116','경기_연천군_106'],
    '서울': ['서울_010'],
}

region_to_station = {
    region: station
    for station, regions in mapping_regions.items()
    for region in regions
}

# ===============================
# 4. 연도별 매핑 처리
# ===============================
for year in [2023, 2024, 2025]:

    print(f"\n===== {year}년 처리 시작 =====")


    # 기상 데이터
    weather_path = f"{weather_dir}OBS_ASOS_DD_{year}.csv"
    df_weather = pd.read_csv(
      weather_path,
      encoding='cp949',
      sep=','
    )

    df_weather['일시'] = pd.to_datetime(df_weather['일시'])

    # 질병 데이터 (연도별 단일 파일)
    disease_path = f"{disease_dir}{year}년_일별__지역별_확진자.csv"
    df_disease = pd.read_csv(
        disease_path,
        encoding='utf-8-sig',
        sep=','
    )
    print("질병컬럼",df_disease.columns)
    print("기상컬럼",df_weather.columns)

    assert df_disease['confirmed_cases'].isna().sum() == 0


    df_disease['date'] = pd.to_datetime(df_disease['date'])



    # 날짜 범위 컷
    dr = date_ranges[year]
    df_disease = df_disease[
        (df_disease['date'] >= dr['start_date']) &
        (df_disease['date'] <= dr['end_date'])
    ]

    # 지역 → 지점명 매핑
    df_disease['지점명'] = df_disease['region'].map(region_to_station)
    df_disease = df_disease.dropna(subset=['지점명'])

    # 병합
    df_merged = pd.merge(
        df_disease,
        df_weather,
        left_on=['date', '지점명'],
        right_on=['일시', '지점명'],
        how='inner'
    )

    # 정렬 (분석 안정성 확보)
    df_merged = df_merged.sort_values(
        by=['date', '지점명', 'disease_subtitle']
    )

    output_path = os.path.join(
        output_dir,
        f"{year}년_일별_지역별_확진자_기상매핑.csv"
    )

    df_merged.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 저장 완료: {output_path}")



===== 2023년 처리 시작 =====
질병컬럼 Index(['date', 'region', 'disease_subtitle', 'confirmed_cases'], dtype='object')
기상컬럼 Index(['지점', '지점명', '일시', '평균기온(°C)', '최저기온(°C)', '최저기온 시각(hhmi)', '최고기온(°C)',
       '최고기온 시각(hhmi)', '강수 계속시간(hr)', '10분 최다 강수량(mm)', '10분 최다강수량 시각(hhmi)',
       '1시간 최다강수량(mm)', '1시간 최다 강수량 시각(hhmi)', '일강수량(mm)', '최대 순간 풍속(m/s)',
       '최대 순간 풍속 풍향(16방위)', '최대 순간풍속 시각(hhmi)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)',
       '최대 풍속 시각(hhmi)', '평균 풍속(m/s)', '풍정합(100m)', '최다풍향(16방위)',
       '평균 이슬점온도(°C)', '최소 상대습도(%)', '최소 상대습도 시각(hhmi)', '평균 상대습도(%)',
       '평균 증기압(hPa)', '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최고 해면기압 시각(hhmi)',
       '최저 해면기압(hPa)', '최저 해면기압 시각(hhmi)', '평균 해면기압(hPa)', '가조시간(hr)',
       '합계 일조시간(hr)', '1시간 최다일사 시각(hhmi)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)',
       '일 최심신적설(cm)', '일 최심신적설 시각(hhmi)', '일 최심적설(cm)', '일 최심적설 시각(hhmi)',
       '합계 3시간 신적설(cm)', '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)',
       '최저 초상온도(°C)', '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)'

## Summary:
***

#### Data Analysis Key Findings  
- 파라미터 초기화 완료: 연도(2023, 2024, 2025), 지역(17개 지정 지역), 연도별 날짜 범위, 기본 디렉터리 경로 등 분석에 필요한 초기 설정은 정상적으로 완료되었다.  
***



## Initialize Parameters for Consolidation

### Subtask:
실제 폴더/파일 구조 확인

가장 우선적으로 /content/drive/MyDrive/AI기상-질병데이터/ 아래의 디렉터리 구조와 파일명을 직접 확인해야 한다.

예:

"/content/drive/MyDrive/AI기상-질병데이터"

연도별 폴더 이름(예: 2023년확진자 vs 2023_년_확진자), 지역 문자열(경기_가평군_075), 파일 접미어 등 경로/이름이 코드와 정확히 일치하는지 점검이 필요하다.​

파일 패턴 기반 자동 탐색 도입

경로를 하드코딩하기보다, glob나 os.walk 등을 활용해 실제 존재하는 CSV 파일 목록을 먼저 읽어오고, 그 결과를 기준으로 연도·지역을 매핑하는 방식으로 개선할 수 있다.​

예:

glob("/content/drive/MyDrive/AI기상-질병데이터/**/*.csv", recursive=True)

이렇게 찾은 경로에서 year, region을 정규식 또는 split로 추출해, 코드의 years/regions와 싱크를 맞추는 방식.

**Reasoning**:
Define the lists for years and regions, and the base input and output directory paths as specified in the subtask.



In [26]:
폐

import pandas as pd
import os

# ===============================
# 1. 경로 설정
# ===============================
disease_dir = '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/'
weather_dir = '/content/drive/MyDrive/AI기상-질병데이터/2023-2025기상데이터/'
output_dir = '/content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/'

os.makedirs(output_dir, exist_ok=True)

# ===============================
# 2. 지역 → 지점명 역매핑 생성
# ===============================
mapping_regions = {
    '동두천': [
        '경기_구리시_082','경기_부천시_오정구_088','경기_과천시_079','경기_부천시_130',
        '경기_광명시_080','경기_부천시_소사구_087','경기_고양시_덕양구_076',
        '경기_의정부시_112','경기_고양시_일산동구_077','경기_고양시_일산서구_078',
        '경기_하남시_117','경기_남양주시_085','경기_김포시_084','경기_양주시_103',
        '경기_파주시_114','경기_동두천시_086',
    ],
    '수원': [
        '경기_수원시_팔달구_096','경기_수원시_장안구_095','경기_수원시_권선구_093',
        '경기_화성시_118','경기_수원시_영통구_094','경기_군포시_083',
        '경기_용인시_기흥구_108','경기_의왕시_111','경기_성남시_수정구_091',
        '경기_안산시_단원구_098','경기_성남시_중원구_092','경기_오산시_107',
        '경기_안산시_상록구_099','경기_안양시_동안구_101','경기_안양시_만안구_102',
        '경기_성남시_분당구_090',
        '경기_용인시_처인구_110','경기_평택시_115',
    ],
    '양평': ['경기_양평군_104','경기_광주시_081','경기_가평군_075'],
    '이천': ['경기_이천시_113','경기_여주시_105','경기_안성시_100'],
    '파주': ['경기_포천시_116','경기_연천군_106'],
    '서울': ['서울_010'],
}

region_to_station = {
    region: station
    for station, regions in mapping_regions.items()
    for region in regions
}

# ===============================
# 3. 연도별 매핑 처리
# ===============================
for year in [2023, 2024, 2025]:

    print(f"\n===== {year}년 처리 시작 =====")

    # 기상 데이터 로드
    weather_path = f"{weather_dir}OBS_ASOS_DD_{year}.csv"
    # Changed encoding to 'cp949' for potentially better Korean character handling
    df_weather = pd.read_csv(weather_path, encoding='cp949')

    df_weather['일시'] = pd.to_datetime(df_weather['일시'])

    # 질병 데이터 파일 순회
    for file in os.listdir(disease_dir):
        if not file.startswith(str(year)):
            continue

        disease_path = os.path.join(disease_dir, file)
        df_disease = pd.read_csv(disease_path, encoding='utf-8')

        df_disease['date'] = pd.to_datetime(df_disease['date'])

        # 지역 → 지점명 매핑
        df_disease['지점명'] = df_disease['region'].map(region_to_station)

        # 지점명이 없는 데이터 제거
        df_disease = df_disease.dropna(subset=['지점명'])

        # 날짜 + 지점명 기준 병합
        df_merged = pd.merge(
            df_disease,
            df_weather,
            left_on=['date', '지점명'],
            right_on=['일시', '지점명'],
            how='inner'
        )

        output_path = os.path.join(output_dir, file)
        df_merged.to_csv(output_path, index=False, encoding='utf-8')

        print(f"매핑 완료: {output_path}")


===== 2023년 처리 시작 =====


KeyboardInterrupt: 

다시 새로운 마음으로 시작

In [28]:
import pandas as pd

# ===============================
# 0. 기본 설정
# ===============================
years = [2023, 2024, 2025]

regions = [
    '경기_가평군_075','경기_고양시_덕양구_076','경기_고양시_일산동구_077',
    '경기_고양시_일산서구_078','경기_과천시_079','경기_광명시_080',
    '경기_광주시_081','경기_구리시_082','경기_군포시_083','경기_김포시_084',
    '경기_남양주시_085','경기_동두천시_086','경기_부천시_130',
    '경기_부천시_소사구_087','경기_부천시_오정구_088',
    '경기_성남시_분당구_090','경기_성남시_수정구_091','경기_성남시_중원구_092',
    '경기_수원시_권선구_093','경기_수원시_영통구_094',
    '경기_수원시_장안구_095','경기_수원시_팔달구_096',
    '경기_시흥시_097','경기_안산시_단원구_098','경기_안산시_상록구_099',
    '경기_안성시_100','경기_안양시_동안구_101','경기_안양시_만안구_102',
    '경기_양주시_103','경기_양평군_104','경기_여주시_105',
    '경기_연천군_106','경기_오산시_107',
    '경기_용인시_기흥구_108','경기_용인시_수지구_109','경기_용인시_처인구_110',
    '경기_의왕시_111','경기_의정부시_112','경기_이천시_113',
    '경기_파주시_114','경기_평택시_115','경기_포천시_116',
    '경기_하남시_117','경기_화성시_118','서울_010'
]

date_ranges = {
    2023: {'start_date': '2023-01-01', 'end_date': '2023-12-30'},
    2024: {'start_date': '2023-12-31', 'end_date': '2024-12-28'},
    2025: {'start_date': '2024-12-29', 'end_date': '2025-12-06'}
}

base_input_dir = '/content/drive/MyDrive/AI기상-질병데이터/'
base_output_dir = '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/'

# ===============================
# 1. 연도별 처리
# ===============================
for year in years:
    yearly_records = []

    for region in regions:
        input_path = f"{base_input_dir}{year}년확진자/kdca_{year}_week_{region}.csv"

        try:
            df = pd.read_csv(input_path, encoding='utf-8-sig')

            year_start = pd.to_datetime(f"{year}-01-01")
            first_sunday = year_start + pd.Timedelta(days=(6 - year_start.weekday()) % 7)

            week_columns = [
                col for col in df.columns
                if col.startswith('COLUMN') and col != 'COLUMN1'
            ]

            for _, row in df.iterrows():
                disease = row['SUBTITLE']

                for week_idx, week_col in enumerate(week_columns):
                    weekly_cases = row[week_col]
                    if pd.isna(weekly_cases):
                        weekly_cases = 0

                    daily_cases = weekly_cases / 7
                    week_start = first_sunday + pd.Timedelta(days=week_idx * 7)

                    for d in range(7):
                        yearly_records.append({
                            'date': week_start + pd.Timedelta(days=d),
                            'region': region,
                            'disease_subtitle': disease,
                            'confirmed_cases': daily_cases
                        })

        except FileNotFoundError:
            print(f"❌ 파일 없음: {input_path}")

    df_year = pd.DataFrame(yearly_records)

    dr = date_ranges[year]
    df_year = df_year[
        (df_year['date'] >= dr['start_date']) &
        (df_year['date'] <= dr['end_date'])
    ]

    df_year = df_year.sort_values(
        ['date', 'region', 'disease_subtitle']
    ).reset_index(drop=True)

    output_path = f"{base_output_dir}{year}년_일별_지역별_확진자.csv"

    # 🔴 인코딩 통일 (중요)
    df_year.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"✅ 저장 완료: {output_path}")


✅ 저장 완료: /content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/2023년_일별_지역별_확진자.csv
✅ 저장 완료: /content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/2024년_일별_지역별_확진자.csv
✅ 저장 완료: /content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/2025년_일별_지역별_확진자.csv


In [29]:
import pandas as pd
import os

disease_dir = '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/'
weather_dir = '/content/drive/MyDrive/AI기상-질병데이터/2023-2025기상데이터/'
output_dir = '/content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/'

os.makedirs(output_dir, exist_ok=True)

date_ranges = {
    2023: {'start_date': '2023-01-01', 'end_date': '2023-12-30'},
    2024: {'start_date': '2023-12-31', 'end_date': '2024-12-28'},
    2025: {'start_date': '2024-12-29', 'end_date': '2025-12-06'}
}

mapping_regions = {
    '동두천': [
        '경기_구리시_082','경기_부천시_오정구_088','경기_과천시_079',
        '경기_부천시_130','경기_광명시_080','경기_부천시_소사구_087',
        '경기_고양시_덕양구_076','경기_의정부시_112',
        '경기_고양시_일산동구_077','경기_고양시_일산서구_078',
        '경기_하남시_117','경기_남양주시_085','경기_김포시_084',
        '경기_양주시_103','경기_파주시_114','경기_동두천시_086'
    ],
    '수원': [
        '경기_수원시_팔달구_096','경기_수원시_장안구_095',
        '경기_수원시_권선구_093','경기_화성시_118',
        '경기_수원시_영통구_094','경기_군포시_083',
        '경기_용인시_기흥구_108','경기_의왕시_111',
        '경기_성남시_수정구_091','경기_안산시_단원구_098',
        '경기_성남시_중원구_092','경기_오산시_107',
        '경기_안산시_상록구_099','경기_안양시_동안구_101',
        '경기_용인시_수지구_109','경기_시흥시_097',
        '경기_안양시_만안구_102','경기_성남시_분당구_090',
        '경기_용인시_처인구_110','경기_평택시_115'
    ],
    '양평': ['경기_양평군_104','경기_광주시_081','경기_가평군_075'],
    '이천': ['경기_이천시_113','경기_여주시_105','경기_안성시_100'],
    '파주': ['경기_포천시_116','경기_연천군_106'],
    '서울': ['서울_010']
}

region_to_station = {
    r: s for s, rs in mapping_regions.items() for r in rs
}

for year in [2023, 2024, 2025]:
    print(f"\n===== {year}년 매핑 =====")

    df_weather = pd.read_csv(
        f"{weather_dir}OBS_ASOS_DD_{year}.csv",
        encoding='cp949'
    )
    df_weather['일시'] = pd.to_datetime(df_weather['일시'])

    df_disease = pd.read_csv(
        f"{disease_dir}{year}년_일별_지역별_확진자.csv",
        encoding='utf-8-sig'
    )
    df_disease['date'] = pd.to_datetime(df_disease['date'])

    dr = date_ranges[year]
    df_disease = df_disease[
        (df_disease['date'] >= dr['start_date']) &
        (df_disease['date'] <= dr['end_date'])
    ]

    df_disease['지점명'] = df_disease['region'].map(region_to_station)
    df_disease = df_disease.dropna(subset=['지점명'])

    df_merge = pd.merge(
        df_disease,
        df_weather,
        left_on=['date', '지점명'],
        right_on=['일시', '지점명'],
        how='inner'
    ).sort_values(['date', '지점명', 'disease_subtitle'])

    output_path = f"{output_dir}{year}년_일별_지역별_확진자_기상매핑.csv"
    df_merge.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"✅ 저장 완료: {output_path}")



===== 2023년 매핑 =====
✅ 저장 완료: /content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/2023년_일별_지역별_확진자_기상매핑.csv

===== 2024년 매핑 =====
✅ 저장 완료: /content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/2024년_일별_지역별_확진자_기상매핑.csv

===== 2025년 매핑 =====
✅ 저장 완료: /content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/2025년_일별_지역별_확진자_기상매핑.csv


In [30]:
climate_numeric_cols = [
'평균기온(°C)',
'최저기온(°C)',
'최고기온(°C)',
'평균 이슬점온도(°C)',
'평균 상대습도(%)',
'최소 상대습도(%)',
'평균 증기압(hPa)',
'일강수량(mm)',
'강수 계속시간(hr)',
'1시간 최다강수량(mm)',
'평균 풍속(m/s)',
'최대 순간 풍속(m/s)',
'합계 일조시간(hr)',
'합계 일사량(MJ/m2)',
'평균 지면온도(°C)',
'안개 계속시간(hr)',
]

In [34]:

import pandas as pd

# ===============================
# 1. 데이터 로드 gpt
# ===============================
base_output_dir = '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/'


input_path = f"{base_output_dir}{year}년_일별_지역별_확진자_기상매핑.csv"

df = pd.read_csv(
    input_path,
    encoding='utf-8-sig'
)

df['date'] = pd.to_datetime(df['date'])

# ===============================
# 2. 기후 수치형 컬럼 정의
# ===============================
climate_numeric_cols = [
    '평균기온(°C)','최저기온(°C)','최고기온(°C)',
    '평균 이슬점온도(°C)','평균 상대습도(%)','최소 상대습도(%)',
    '평균 증기압(hPa)','일강수량(mm)','강수 계속시간(hr)',
    '1시간 최다강수량(mm)','평균 풍속(m/s)','최대 순간 풍속(m/s)',
    '합계 일조시간(hr)','합계 일사량(MJ/m2)',
    '평균 지면온도(°C)','안개 계속시간(hr)'
]

# ===============================
# 3. 타입 정제 (필수)
# ===============================
for col in climate_numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# ===============================
# 4. 정렬 (시계열 feature 필수 조건)
# ===============================
df = df.sort_values(
    by=['지점명', 'date']
).reset_index(drop=True)

# ===============================
# 5. 결측치 처리 (확정 전략)
# ===============================
# 지점별 시계열 기준 선형 보간
df[climate_numeric_cols] = (
    df.groupby('지점명')[climate_numeric_cols]
      .apply(lambda x: x.interpolate(method='linear'))
)

# ===============================
# 6. Lag Feature 생성
# ===============================
lag_days = [1, 3, 7, 14]

for lag in lag_days:
    for col in climate_numeric_cols:
        df[f'{col}_lag{lag}'] = (
            df.groupby('지점명')[col].shift(lag)
        )

# ===============================
# 7. Rolling Feature 생성
# ===============================
rolling_windows = [3, 7, 14]

for window in rolling_windows:
    for col in climate_numeric_cols:
        df[f'{col}_roll{window}'] = (
            df.groupby('지점명')[col]
              .rolling(window)
              .mean()
              .reset_index(level=0, drop=True)
        )

# ===============================
# 8. 결측치 최종 정리
# ===============================
df = df.dropna().reset_index(drop=True)

# ===============================
# 9. 저장
# ===============================
output_path = '2023년_feature_engineered_dataset.csv'

df.to_csv(
    output_path,
    index=False,
    encoding='utf-8-sig'
)

print(f"✅ Feature Engineering 완료: {output_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별로 전처리/2023년_일별_지역별_확진자_기상매핑.csv'

In [41]:
import pandas as pd
import os

# Define output_dir to load from and save to
output_dir = '/content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

years = [2023, 2024, 2025]

# ===============================
# 2. 기후 수치형 컬럼 정의
# ===============================
climate_numeric_cols = [
    '평균기온(°C)','최저기온(°C)','최고기온(°C)',
    '평균 이슬점온도(°C)','평균 상대습도(%)','최소 상대습도(%)',
    '평균 증기압(hPa)','일강수량(mm)','강수 계속시간(hr)',
    '1시간 최다강수량(mm)','평균 풍속(m/s)','최대 순간 풍속(m/s)',
    '합계 일조시간(hr)','합계 일사량(MJ/m2)',
    '평균 지면온도(°C)','안개 계속시간(hr)'
]

for year in years:
    print(f"\n===== {year}년 Feature Engineering 시작 =====")

    # ===============================
    # 1. 데이터 로드 (매핑된 데이터)
    # ===============================
    input_path = f"{output_dir}{year}년_일별_지역별_확진자_기상매핑.csv"

    df = pd.read_csv(
        input_path,
        encoding='utf-8-sig'
    )

    df['date'] = pd.to_datetime(df['date'])

    # ===============================
    # 3. 타입 정제 (필수)
    # ===============================
    for col in climate_numeric_cols:
        # Check if the column exists before trying to convert
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            print(f"Warning: Column '{col}' not found in DataFrame for {year}.")

    # ===============================
    # 4. 정렬 (시계열 feature 필수 조건)
    # ===============================
    df = df.sort_values(
        by=['지점명', 'date']
    ).reset_index(drop=True)

    # ===============================
    # 5. 결측치 처리 (확정 전략)
    # ===============================
    # 지점별 시계열 기준 선형 보간
    # Filter to only existing climate numeric columns before applying interpolation
    existing_climate_cols = [col for col in climate_numeric_cols if col in df.columns]
    if existing_climate_cols:
        # Changed .apply(lambda x: x.interpolate(method='linear')) to .transform(lambda x: x.interpolate(method='linear'))
        df[existing_climate_cols] = (
            df.groupby('지점명')[existing_climate_cols]
              .transform(lambda x: x.interpolate(method='linear'))
        )
    else:
        print(f"No existing climate numeric columns to interpolate for {year}.")


    # ===============================
    # 6. Lag Feature 생성
    # ===============================
    lag_days = [1, 3, 7, 14]

    for lag in lag_days:
        for col in existing_climate_cols:
            df[f'{col}_lag{lag}'] = (
                df.groupby('지점명')[col].shift(lag)
            )

    # ===============================
    # 7. Rolling Feature 생성
    # ===============================
    rolling_windows = [3, 7, 14]

    for window in rolling_windows:
        for col in existing_climate_cols:
            df[f'{col}_roll{window}'] = (
                df.groupby('지점명')[col]
                  .rolling(window)
                  .mean()
                  .reset_index(level=0, drop=True)
            )

    # ===============================
    # 8. 결측치 최종 정리
    # ===============================
    # Drop rows with NaN values created by shift/rolling operations
    df = df.dropna().reset_index(drop=True)

    # ===============================
    # 9. 저장
# ===============================
    output_file_path = f"{output_dir}{year}년_feature_engineered_dataset.csv"

    df.to_csv(
        output_file_path,
        index=False,
        encoding='utf-8-sig'
    )

    print(f"✅ Feature Engineering 완료 및 저장: {output_file_path}")


===== 2023년 Feature Engineering 시작 =====


/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{window}'] = (
/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{window}'] = (
/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.co

✅ Feature Engineering 완료 및 저장: /content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/2023년_feature_engineered_dataset.csv

===== 2024년 Feature Engineering 시작 =====


/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{window}'] = (
/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{window}'] = (
/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.co

✅ Feature Engineering 완료 및 저장: /content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/2024년_feature_engineered_dataset.csv

===== 2025년 Feature Engineering 시작 =====


/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{window}'] = (
/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{window}'] = (
/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.co

✅ Feature Engineering 완료 및 저장: /content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/2025년_feature_engineered_dataset.csv


/tmp/ipython-input-1421773537.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{window}'] = (


In [44]:
import pandas as pd

# ===============================
# 0. 기본 설정
# ===============================
years = [2023, 2024, 2025]

input_dir = '/content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/'
output_dir = '/content/drive/MyDrive/AI기상-질병데이터/feature_dataset/'

import os
os.makedirs(output_dir, exist_ok=True)

# ===============================
# 1. 기후 수치형 컬럼
# ===============================
climate_numeric_cols = [
    '평균기온(°C)','최저기온(°C)','최고기온(°C)',
    '평균 이슬점온도(°C)','평균 상대습도(%)','최소 상대습도(%)',
    '평균 증기압(hPa)','일강수량(mm)','강수 계속시간(hr)',
    '1시간 최다강수량(mm)','평균 풍속(m/s)','최대 순간 풍속(m/s)',
    '합계 일조시간(hr)','합계 일사량(MJ/m2)',
    '평균 지면온도(°C)','안개 계속시간(hr)'
]

# ===============================
# 2. 연도별 반복 처리
# ===============================
for year in years:
    print(f"\n===== {year}년 Feature Engineering 시작 ====")

    input_path = f"{input_dir}{year}년_일별_지역별_확진자_기상매핑.csv"

    df = pd.read_csv(
        input_path,
        encoding='utf-8-sig'
    )

    df['date'] = pd.to_datetime(df['date'])

    print("▶ DataFrame shape:", df.shape)
    print("▶ Columns (앞 10개):")
    print(df.columns.tolist()[:10])
    print("▶ Index type:", type(df.index))


    # -------------------------------
    # 컬럼 존재 여부 검증 (안전장치)
    # -------------------------------
    missing_cols = [c for c in climate_numeric_cols if c not in df.columns]
    if missing_cols:
        raise ValueError(f"{year}년 데이터에 기후 컬럼 없음: {missing_cols}")

    print("\n[CHECK] climate_numeric_cols 존재 여부")

    missing_cols = [c for c in climate_numeric_cols if c not in df.columns]

    if missing_cols:
        print("❌ 존재하지 않는 컬럼:")
        for c in missing_cols:
            print("   -", repr(c))
    else:
        print("✅ 모든 climate 컬럼 존재")


    # -------------------------------
    # 타입 정제
    # -------------------------------

    print("\n[CHECK] climate 컬럼 dtype")

    print(df[climate_numeric_cols].dtypes)

    for col in climate_numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # -------------------------------
    # 정렬 (지점별 시계열) - interpolate 전에 정렬 필수
    # -------------------------------
    df = df.sort_values(
        by=['지점명', 'date']
    ).reset_index(drop=True)

    # -------------------------------
    # 결측치 처리 (지점별 선형 보간)
    # -------------------------------
    df[climate_numeric_cols] = (
    df.groupby('지점명')[climate_numeric_cols]
      .transform(lambda x: x.interpolate(method='linear'))
    )

    tmp = (
        df.groupby('지점명')[climate_numeric_cols]
          .transform(lambda x: x.interpolate(method='linear'))
    )

    print("\n[CHECK] interpolate 결과")
    print(" - type:", type(tmp))
    print(" - shape:", tmp.shape)
    print(" - index equals df.index ?", tmp.index.equals(df.index))


    # -------------------------------
    # Lag Feature
    # -------------------------------
    lag_days = [1, 3, 7, 14]
    lag_features = [] # List to store new lag features

    for lag in lag_days:
        for col in climate_numeric_cols:
            lag_features.append(
                df.groupby('지점명')[col].shift(lag).rename(f'{col}_lag{lag}')
            )

    # Concatenate all lag features at once
    if lag_features:
        df = pd.concat([df] + lag_features, axis=1)

    col = climate_numeric_cols[0]

    tmp_lag = (
        df.groupby('지점명')[col]
          .transform(lambda x: x.shift(1))
    )

    print("\n[CHECK] lag feature")
    print(" - type:", type(tmp_lag))
    print(" - length:", len(tmp_lag))
    print(" - index equals df.index ?", tmp_lag.index.equals(df.index))

    # -------------------------------
    # Rolling Feature
    # -------------------------------
    rolling_windows = [3, 7, 14]
    rolling_features = [] # List to store new rolling features

    for window in rolling_windows:
        for col in climate_numeric_cols:
            rolling_features.append(
                df.groupby('지점명')[col]
                  .rolling(window)
                  .mean()
                  .reset_index(level=0, drop=True)
                  .rename(f'{col}_roll{window}')
            )

    # Concatenate all rolling features at once
    if rolling_features:
        df = pd.concat([df] + rolling_features, axis=1)

    # -------------------------------
    # 결측 제거
    # -------------------------------
    df = df.dropna().reset_index(drop=True)

    tmp_roll = (
        df.groupby('지점명')[col]
          .transform(lambda x: x.rolling(7).mean())
    )

    print("\n[CHECK] rolling feature")
    print(" - type:", type(tmp_roll))
    print(" - length:", len(tmp_roll))
    print(" - index equals df.index ?", tmp_roll.index.equals(df.index))

    # -------------------------------
    # 저장
    # -------------------------------
    output_path = f"{output_dir}{year}년_feature_engineered_dataset.csv"

    df.to_csv(
        output_path,
        index=False,
        encoding='utf-8-sig'
    )

    print(f"✅ {year}년 Feature Engineering 완료")


===== 2023년 Feature Engineering 시작 ====
▶ DataFrame shape: (212940, 66)
▶ Columns (앞 10개):
['date', 'region', 'disease_subtitle', 'confirmed_cases', '지점명', '지점', '일시', '평균기온(°C)', '최저기온(°C)', '최저기온 시각(hhmi)']
▶ Index type: <class 'pandas.core.indexes.range.RangeIndex'>

[CHECK] climate_numeric_cols 존재 여부
✅ 모든 climate 컬럼 존재

[CHECK] climate 컬럼 dtype
평균기온(°C)         float64
최저기온(°C)         float64
최고기온(°C)         float64
평균 이슬점온도(°C)     float64
평균 상대습도(%)       float64
최소 상대습도(%)       float64
평균 증기압(hPa)      float64
일강수량(mm)         float64
강수 계속시간(hr)      float64
1시간 최다강수량(mm)    float64
평균 풍속(m/s)       float64
최대 순간 풍속(m/s)    float64
합계 일조시간(hr)      float64
합계 일사량(MJ/m2)    float64
평균 지면온도(°C)      float64
안개 계속시간(hr)      float64
dtype: object

[CHECK] interpolate 결과
 - type: <class 'pandas.core.frame.DataFrame'>
 - shape: (212940, 16)
 - index equals df.index ? True

[CHECK] lag feature
 - type: <class 'pandas.core.series.Series'>
 - length: 212940
 - index equals df.index